<a href="https://colab.research.google.com/github/Max-FM/SPRINT-Colombia/blob/main/Visualising_Extreme_Weather_Events_Using_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Visualising Extreme Weather Events Using Google Earth Engine 

##Install required packages

In [5]:
%%capture

!pip install geemap
!pip install branca

##Import required packages

In [17]:
import ee
import folium
import pandas as pd
import branca.colormap as cm
import geemap.eefolium as emap

##Authenticate Google Earth Engine

To access the Google Earth Engine API you require an account. To request access, go to [https://signup.earthengine.google.com](https://signup.earthengine.google.com/). You may have to wait up to a day or so to be granted access and it's possible you will not recieve any email communication. To manually check whether you have access, try to log into [https://code.earthengine.google.com](https://code.earthengine.google.com/), or attempt to run the next cell and follow the instructions provided in the output cell.

In [7]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=8HrHNzU62glwR_PoVsS9152pjrAQuvDh869howD0xGo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/5wEdb9ukzQeN9JHmzk4qpQBmQ933Ec7fL9iUUbXE2qJSxvfTiTu05l8

Successfully saved authorization token.


##Define Request Function

In [8]:
def obtain_data(region, start_date, end_date, months_either_side=0, max_cloud_cover=80):
    start_date = ee.Date(start_date)
    start_date = start_date.advance(-months_either_side, 'month')

    end_date = ee.Date(end_date)
    end_date = end_date.advance(months_either_side, 'month')
    
    # Filter input collections by desired date range, region and cloud coverage.
    criteria  = ee.Filter.And(ee.Filter.geometry(region), 
                              ee.Filter.date(start_date, end_date))
    
    Sentinel_2_SR = ee.ImageCollection('COPERNICUS/S2_SR') \
                      .filter(criteria) \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)) \
                      .select(['B4', 'B3', 'B2', 'B8'])

    Landsat_8_T1_SR = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
                        .filter(criteria) \
                        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover)) \
                        .select(['B4', 'B3', 'B2', 'B5'])

    Landsat_7_T1_SR = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") \
                        .filter(criteria) \
                        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover)) \
                        .select(['B3', 'B2', 'B1', 'B4'])

    Landsat_5_T1_SR = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
                        .filter(criteria) \
                        .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover)) \
                        .select(['B3', 'B2', 'B1', 'B4'])

    MODIS_16D_NDVI = ee.ImageCollection("MODIS/006/MOD13Q1") \
                       .filter(criteria) \
                       .select('NDVI')

    image_collections = {'Sentinel_2_SR': Sentinel_2_SR,
                         'Landsat_8_T1_SR': Landsat_8_T1_SR,
                         'Landsat_7_T1_SR': Landsat_7_T1_SR,
                         'Landsat_5_T1_SR': Landsat_5_T1_SR,
                         'MODIS_16D_NDVI': MODIS_16D_NDVI}

    return image_collections

## Importing Table of Disaster Dates

In [18]:
disaster_dates = pd.read_csv('/content/drive/Shared drives/Colombia SPRINT/Test Districts/disaster_dates.csv', index_col=0)

disaster_dates

,Disaster,Start_Datetime,End_Datetime
0,Drought,1998-01-01,1999-01-01
1,Flood,1999-01-10,1999-05-19
2,Earthquake,1999-01-25,1999-01-26
3,Flood,1999-10-28,1999-12-31
4,Flood,2000-05-18,2000-05-24
5,Wildfire,2001-08-01,2001-09-01
6,Drought,2002-01-01,2003-01-01
7,Flood,2002-04-24,2002-04-29
8,Flood,2003-08-01,2003-12-01
9,Flood,2004-01-01,2004-06-28


##Visualise Imaging for All Events

In [25]:
test_districts = {'Dosquebradas': ee.FeatureCollection('users/maxfoxley-marrable/SPRINT/Dosquebradas'),
                  'Versailles': ee.FeatureCollection('users/maxfoxley-marrable/SPRINT/Versailles')}

scale_dict = {'Sentinel_2_SR': 10,
              'Sentinel_1_SAR_GRD_C_BAND': 10,
              'Landsat_8_T1_SR': 30,
              'Landsat_7_T1_SR': 30,
              'Landsat_5_T1_SR': 30,
              'MODIS_16D_NDVI': 250}

bands_dict = {'Sentinel_2_SR': ['B4', 'B3', 'B2'], 
              'Landsat_8_T1_SR': ['B4', 'B3', 'B2'],
              'Landsat_7_T1_SR': ['B3', 'B2', 'B1'],
              'Landsat_5_T1_SR': ['B3', 'B2', 'B1'],
              'MODIS_16D_NDVI': 'NDVI'} 

##Testing the Code for the Latest Event

In [26]:
districts = ee.Geometry.Rectangle([[-75.63431636299883,4.546918365359551],
                                   [-76.23856440987383,4.905496619857987]])

desired_collections = None

disaster = disaster_dates.iloc[19]

disaster_type = disaster['Disaster']
start_date = disaster['Start_Datetime']
end_date = disaster['End_Datetime']

display(f'{disaster_type}: {start_date} - {end_date}')

Map = emap.Map()
Map.center_object(districts, zoom=11)

image_collections = obtain_data(districts, 
                                start_date, 
                                end_date,
                                months_either_side=2,
                                max_cloud_cover=80)

if disaster_type == 'Drought':
    # Probably only want MODIS 16D imagery for El Nino droughts.
    image_collections = {'MODIS_16D_NDVI': image_collections['MODIS_16D_NDVI']}
        
# Filters out unwanted collections if defined above.
elif desired_collections:
    image_collections = {collection: image_collections[collection] \
                            for collection in desired_collections} 

for collection_name, collection in image_collections.items():

    if collection.size().getInfo() == 0:
        continue

    collectionList = collection.toList(collection.size())
    collectionSize = collectionList.size().getInfo()
    
    if collection_name == 'MODIS_16D_NDVI':
        vis_params = {'min': 0,
                        'max': 8000,
                        'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                                    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                                    '012E01', '011D01', '011301']}
    else:
        vis_params = {'min': 0, 'max': 3000, 'bands': bands_dict[collection_name]}

    for i in range(collectionSize):
        image = ee.Image(collectionList.get(i))
        time = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        Map.addLayer(image, name=f'{collection_name} ({time})', vis_params=vis_params, shown=False)

Map.addLayer(test_districts['Dosquebradas'], name='Dosquebradas', vis_params={'color':'red'})
Map.addLayer(test_districts['Versailles'], name='Versailles', vis_params={'color':'red'})
Map.add_layer_control()

colours = ['#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718', 
            '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201', 
            '#004C00', '#023B01', '#012E01', '#011D01', '#011301']

colourbar = cm.LinearColormap(colours, vmin=0, vmax=0.8, caption='NDVI')
Map.add_child(colourbar, name='NDVI')

display(Map)

'Wildfire: 2010-01-01 - 2010-04-06'

## Creating and Exporting Interactive Maps for Chosen Weather Events

Exported interactive maps unfortunately expire after few days, this is due to an authentication issue with Google Earth Engine. This requires fixing for the interactive maps to work permenantley.

In [29]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 20000})'''))

districts = ee.Geometry.Rectangle([[-75.63431636299883,4.546918365359551],
                                   [-76.23856440987383,4.905496619857987]])

desired_collections = None

for index, disaster in disaster_dates.iterrows():

    disaster_type = disaster['Disaster']
    start_date = disaster['Start_Datetime']
    end_date = disaster['End_Datetime']

    display(f'{disaster_type}: {start_date} - {end_date}')

    Map = emap.Map()
    Map.center_object(districts, zoom=11)

    image_collections = obtain_data(districts, 
                                    start_date, 
                                    end_date,
                                    months_either_side=2,
                                    max_cloud_cover=80)
    
    if disaster_type == 'Drought':
        # Probably only want MODIS 16D imagery for El Nino droughts.
        image_collections = {'MODIS_16D_NDVI': image_collections['MODIS_16D_NDVI']}
        
    # Filters out unwanted collections if defined above.
    elif desired_collections:
        image_collections = {collection: image_collections[collection] \
                                for collection in desired_collections} 

    for collection_name, collection in image_collections.items():

        if collection.size().getInfo() == 0:
            continue

        collectionList = collection.toList(collection.size())
        collectionSize = collectionList.size().getInfo()
        
        if collection_name == 'MODIS_16D_NDVI':
            vis_params = {'min': 0,
                          'max': 8000,
                          'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                                      '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                                      '012E01', '011D01', '011301']}
        else:
            vis_params = {'min': 0, 'max': 3000, 'bands': bands_dict[collection_name]}

        for i in range(collectionSize):
            image = ee.Image(collectionList.get(i))
            time = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            Map.addLayer(image, name=f'{collection_name} ({time})', vis_params=vis_params, shown=False)

    Map.addLayer(test_districts['Dosquebradas'], name='Dosquebradas', vis_params={'color':'red'})
    Map.addLayer(test_districts['Versailles'], name='Versailles', vis_params={'color':'red'})
    Map.add_layer_control()

    colours = ['#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718', 
               '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201', 
               '#004C00', '#023B01', '#012E01', '#011D01', '#011301']

    colourbar = cm.LinearColormap(colours, vmin=0, vmax=0.8, caption='NDVI')
    Map.add_child(colourbar, name='NDVI')

    # display(Map)
    
    Map.save(f'/content/drive/Shared drives/Colombia SPRINT/Test Districts/Interactive Maps/{disaster_type}_{start_date}_{end_date}.html')

<IPython.core.display.Javascript object>

'Drought: 1998-01-01 - 1999-01-01'

'Flood: 1999-01-10 - 1999-05-19'

'Earthquake: 1999-01-25 - 1999-01-26'

'Flood: 1999-10-28 - 1999-12-31'

...

'Flood: 2017-03-17 - 2017-05-16'

'Flood: 2017-12-01 - 2018-01-07'

'Drought: 2018-01-01 - 2020-01-01'

'Flood: 2019-02-20 - 2019-02-26'

'Flood: 2020-06-10 - 2020-07-10'